Duvall Pinkney green taxi dataset 

# Homework 15

This homework uses the Green taxi data from Homework 2 and 14.  The 2020 green taxi trip data from NYC Open Data was filtered to only be trips with a pickup date and time on Feb. 4, 2020. 

The `RatecodeID` column is a qualitative column with the following categories:
* 1 = Standard rate
* 2 = JFK (flat rate from JFK to Manhattan)
* 3 = Newark
* 4 = Nassau or Westchester
* 5 = Negotiated fare
* 6 = Group ride

Filtered trip data URL: [https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb4_2020_Green_Taxi_Trip_Data.csv](https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb4_2020_Green_Taxi_Trip_Data.csv)

We will try to predict which trips involve tolls using the `RatecodeId`, `passenger_count`, `trip_distance`, and `total_amount` columns.

**Answer all questions in the homework, including ones that require written answers.**  For questions related to Homework 14, you can use the Homework 14 solutions posted on Blackboard if you did not complete Homework 14.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf
from scipy.special import expit
from scipy.stats import logistic

%matplotlib inline

### Question 1: Loading and cleaning the data

1a) Load the data into Python.

In [4]:
url = "https://raw.githubusercontent.com/megan-owen/MAT328-Techniques_in_Data_Science/main/data/Feb4_2020_Green_Taxi_Trip_Data.csv"
df = pd.read_csv(url)
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,02/04/2020 12:00:07 AM,02/04/2020 12:08:01 AM,N,1.0,42,152,1.0,1.21,7.00,0.50,0.5,0.00,0.0,NaN,0.3,8.30,1.0,1.0,0.0
1,2.0,02/04/2020 12:00:20 AM,02/04/2020 12:13:44 AM,N,1.0,25,188,1.0,3.28,12.50,0.50,0.5,0.00,0.0,NaN,0.3,13.80,2.0,1.0,0.0
2,2.0,02/04/2020 12:00:42 AM,02/04/2020 12:03:29 AM,N,1.0,166,152,1.0,0.68,4.00,0.50,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0
3,2.0,02/04/2020 12:01:00 AM,02/04/2020 12:13:15 AM,N,1.0,82,173,1.0,1.89,9.50,0.50,0.5,1.50,0.0,NaN,0.3,12.30,1.0,1.0,0.0
4,NaN,02/04/2020 12:01:00 AM,02/04/2020 12:31:00 AM,NaN,NaN,136,90,NaN,12.80,51.55,2.75,0.0,0.00,0.0,NaN,0.3,54.60,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13976,2.0,02/04/2020 11:57:53 PM,02/05/2020 12:01:00 AM,N,1.0,112,255,1.0,0.64,4.50,0.50,0.5,0.00,0.0,NaN,0.3,5.80,2.0,1.0,0.0
13977,2.0,02/04/2020 11:58:14 PM,02/05/2020 12:09:00 AM,N,1.0,97,52,1.0,1.77,9.00,0.50,0.5,0.00,0.0,NaN,0.3,10.30,1.0,1.0,0.0
13978,2.0,02/04/2020 11:58:29 PM,02/05/2020 12:05:29 AM,N,1.0,25,181,1.0,1.80,7.50,0.50,0.5,2.64,0.0,NaN,0.3,11.44,1.0,1.0,0.0
13979,2.0,02/04/2020 11:59:37 PM,02/05/2020 12:02:39 AM,N,1.0,7,7,1.0,0.69,4.50,0.50,0.5,0.00,0.0,NaN,0.3,5.80,2.0,1.0,0.0


Clean the data as follows:

1b) Create a new column call `paid_toll` that contains a 1 if the value in the `tolls_amount` column is greater than 0 and a 0 otherwise.  Hint:  See Lab 1, Section 6.

In [5]:
def paid_toll(x):
    if x > 0:
        return 1
    else:
        return 0

In [10]:
df["paid_toll"] = df["tolls_amount"].apply(paid_toll)

In [12]:
df2 = df[["trip_distance","paid_toll"]]
df2

,trip_distance,paid_toll
0,1.21,0
1,3.28,0
2,0.68,0
3,1.89,0
4,12.80,0
...,...,...
13976,0.64,0
13977,1.77,0
13978,1.80,0
13979,0.69,0


1c) Create a new DataFrame (or change your current DataFrame) that only contains the columns `RatecodeID`, `passenger_count`,`trip_distance`, `total_amount`, and `paid_toll`

In [17]:
df3 = df[["RatecodeID","passenger_count","trip_distance","total_amount","paid_toll"]]
df3

,RatecodeID,passenger_count,trip_distance,total_amount,paid_toll
0,1.0,1.0,1.21,8.30,0
1,1.0,1.0,3.28,13.80,0
2,1.0,1.0,0.68,5.30,0
3,1.0,1.0,1.89,12.30,0
4,NaN,NaN,12.80,54.60,0
...,...,...,...,...,...
13976,1.0,1.0,0.64,5.80,0
13977,1.0,1.0,1.77,10.30,0
13978,1.0,1.0,1.80,11.44,0
13979,1.0,1.0,0.69,5.80,0


1d) Do any of the rows in the DataFrame created in question 1c have missing values?  If so, drop all rows with at least one NaN values.   How many rows did you drop?

In [18]:
df4 = df3.dropna()
df4

,RatecodeID,passenger_count,trip_distance,total_amount,paid_toll
0,1.0,1.0,1.21,8.30,0
1,1.0,1.0,3.28,13.80,0
2,1.0,1.0,0.68,5.30,0
3,1.0,1.0,1.89,12.30,0
5,1.0,2.0,4.57,17.30,0
...,...,...,...,...,...
13976,1.0,1.0,0.64,5.80,0
13977,1.0,1.0,1.77,10.30,0
13978,1.0,1.0,1.80,11.44,0
13979,1.0,1.0,0.69,5.80,0


1d) Change the type of the `RatecodeID` column to `int` to make referencing the dummy variables easier, and then replace the `RatecodeID` column with dummy variables for it  Hint: See lab 4, Section 5 for changing the type of the column.


In [20]:
df4["RatecodeID"] = df4["RatecodeID"].astype(int)
df4

C:\Users\thees\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,RatecodeID,passenger_count,trip_distance,total_amount,paid_toll
0,1,1.0,1.21,8.30,0
1,1,1.0,3.28,13.80,0
2,1,1.0,0.68,5.30,0
3,1,1.0,1.89,12.30,0
5,1,2.0,4.57,17.30,0
...,...,...,...,...,...
13976,1,1.0,0.64,5.80,0
13977,1,1.0,1.77,10.30,0
13978,1,1.0,1.80,11.44,0
13979,1,1.0,0.69,5.80,0


### Question 2: Logistic regression

2a) Create a logistic model where `paid_toll` is the dependent variable, and all other columns in your final, cleaned DataFrame from Question 1 are the independent variables.

2b) Recall that the p-value for each variable (in the column P > |z|) is the probability that we would get the coefficient in the `coef` column or one with a greater magnitude, when that coefficient is really 0 (meaning that variable does not affect the dependent variable).  Make a new logistic model with only those variables with a p-value less than 0.05, and compute the summary.

2c)  What is the Pseudo R-squared value for your new logistic model?  What does it suggest about the fit of the model?  How does this compare to the pseudo r-squared value for all your first logistic regression model with all the variables?  How does it compare to the logistic regression model from Homework 14 that only used `trip_distance` as the independent variable?

2d)  What is the equation of your logistic regression model?  The equation below is written in Latex, which is a system for typing math equations.  Double click on this cell, and replace the part of the equation that says "reg" to finish the formula.  "run" this cell (as you would a code cell) when you are finished.

$$y = \frac{1}{1 + e^{-(reg)}}$$

2f) Compute the confusion matrix for your second logistic regression model.  What is the accuracy, sensitivity, specificity, and precision of your model?  What does mean the strengths and weaknesses of the model are?